# Libraries

In [1]:
# for data manipulation
import numpy as np
import pandas as pd

# our defined functions for data preparation
import utils.functions as ft

# setting option
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Load Data

In [2]:
bu_raw = pd.read_csv(r"Data/dseb63_bureau.csv")
bu_raw = bu_raw[['SK_ID_CURR']+list(bu_raw.columns[:-1])]
print("Bureau dataset shape:", bu_raw.shape)
bu_raw.head()

Bureau dataset shape: (1465325, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,254629,5714462,Closed,currency 1,-497,0,-153.00000,-153.00000,NaN,0,91323.00000,0.00000,NaN,0.00000,Consumer credit,-131,NaN
1,254629,5714463,Active,currency 1,-208,0,1075.00000,NaN,NaN,0,225000.00000,171342.00000,NaN,0.00000,Credit card,-20,NaN
2,254629,5714464,Active,currency 1,-203,0,528.00000,NaN,NaN,0,464323.50000,NaN,NaN,0.00000,Consumer credit,-16,NaN
3,254629,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00000,NaN,NaN,0.00000,Credit card,-16,NaN
4,254629,5714466,Active,currency 1,-629,0,1197.00000,NaN,77674.50000,0,2700000.00000,NaN,NaN,0.00000,Consumer credit,-21,NaN


In [3]:
bubal_raw = pd.read_csv(r"Data/dseb63_bureau_balance.csv")
print("Bureau dataset shape:", bubal_raw.shape)
bubal_raw.head()

Bureau dataset shape: (27299925, 3)


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [5]:
# file contains only SK_ID_CURR and TARGET from train_test (defined in DP_TRAIN_TEST notebook)
id_target = pd.read_csv("Data/dseb63_id_target.csv", index_col=0)
id_target.head()

,SK_ID_CURR,TARGET
0,278621,0.00000
1,139008,0.00000
2,138348,0.00000
3,64140,0.00000
4,219374,0.00000


# Data Preparation

### bureau_balance

__1. Data Exploration__

In [6]:
bu = bu_raw.copy()

In [7]:
ft.data_explore(bu)

,info
Rows,1465325
Features,17
Duplicate Rows,0
float64,8
int64,6
object,3


In [8]:
ft.check_nan(bu)

,nan,%nan
AMT_ANNUITY,1130013,77.11689
AMT_CREDIT_MAX_OVERDUE,948545,64.73274
DAYS_ENDDATE_FACT,544673,37.17080
AMT_CREDIT_SUM_LIMIT,489670,33.41716
AMT_CREDIT_SUM_DEBT,223094,15.22488
DAYS_CREDIT_ENDDATE,89098,6.08043
AMT_CREDIT_SUM,3,0.00020


In [9]:
ft.multi_features_explore(bu)

index,dtype,nonnull,%nonnull,nan,%nan,nunique,nunique_nan,max,min,mean,std,unique,frequency,%value,most
SK_ID_CURR,int64,1465325,1.00000,0,0.00000,263491,263491,307509,0,153876.24812,88898.87845,NaN,NaN,NaN,NaN
SK_ID_BUREAU,int64,1465325,1.00000,0,0.00000,1465325,1465325,6843457,5000000,5890327.73856,537827.07116,NaN,NaN,NaN,NaN
CREDIT_ACTIVE,object,1465325,1.00000,0,0.00000,4,4,NaN,NaN,NaN,NaN,"[Closed, Active, Sold, Bad debt]","{'Closed': 917733, 'Active': 541919, 'Sold': 5653, 'Bad debt': 20}","{'Closed': 0.63, 'Active': 0.37, 'Sold': 0.0, 'Bad debt': 0.0}",[Closed]
CREDIT_CURRENCY,object,1465325,1.00000,0,0.00000,4,4,NaN,NaN,NaN,NaN,"[currency 1, currency 2, currency 4, currency 3]","{'currency 1': 1464094, 'currency 2': 1072, 'currency 3': 150, 'currency 4': 9}","{'currency 1': 1.0, 'currency 2': 0.0, 'currency 3': 0.0, 'currency 4': 0.0}",[currency 1]
DAYS_CREDIT,int64,1465325,1.00000,0,0.00000,2923,2923,0,-2922,-1143.60041,800.02005,NaN,NaN,NaN,NaN
CREDIT_DAY_OVERDUE,int64,1465325,1.00000,0,0.00000,893,893,2792,0,0.88876,38.24707,NaN,NaN,NaN,NaN
DAYS_CREDIT_ENDDATE,float64,1376227,0.94000,89098,0.06000,13641,13642,31199.00000,-42060.00000,493.13220,4964.90531,NaN,NaN,NaN,NaN
DAYS_ENDDATE_FACT,float64,920652,0.63000,544673,0.37000,2916,2917,0.00000,-42023.00000,-1022.76529,718.37175,NaN,NaN,NaN,NaN
AMT_CREDIT_MAX_OVERDUE,float64,516780,0.35000,948545,0.65000,61066,61067,115987185.00000,0.00000,3942.85214,219685.63451,NaN,NaN,NaN,NaN
CNT_CREDIT_PROLONG,int64,1465325,1.00000,0,0.00000,10,10,9,0,0.00728,0.10264,"[0, 2, 1, 4, 3, 5, 6, 7, 9, 8]","{0: 1456505, 1: 7363, 2: 1189, 3: 189, 4: 53, 5: 20, 6: 2, 9: 2, 7: 1, 8: 1}","{0: 0.99, 1: 0.01, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 9: 0.0, 7: 0.0, 8: 0.0}",NaN


__2. Create New Features__

In [11]:
# flag if there is day past due
bu['BU_FLAG_DPD'] = bu['CREDIT_DAY_OVERDUE'].apply(lambda x: 0 if x==0 else 1)

# flag if that application had been prolonged at least one time
bu['FLAG_PROLONG'] = bu['CNT_CREDIT_PROLONG'].apply(lambda x: 0 if x==0 else 1)

__3. Handling categorical features__

In [16]:
bu_cat = ft.sub_cate_norm(bu,'SK_ID_CURR', 'BUR').reset_index()
bu_cat.head()

,SK_ID_CURR,BUR_CREDIT_ACTIVE_Active_count,BUR_CREDIT_ACTIVE_Active_norm,BUR_CREDIT_ACTIVE_Bad debt_count,BUR_CREDIT_ACTIVE_Bad debt_norm,BUR_CREDIT_ACTIVE_Closed_count,BUR_CREDIT_ACTIVE_Closed_norm,BUR_CREDIT_ACTIVE_Sold_count,BUR_CREDIT_ACTIVE_Sold_norm,BUR_CREDIT_CURRENCY_currency 1_count,BUR_CREDIT_CURRENCY_currency 1_norm,BUR_CREDIT_CURRENCY_currency 2_count,BUR_CREDIT_CURRENCY_currency 2_norm,BUR_CREDIT_CURRENCY_currency 3_count,BUR_CREDIT_CURRENCY_currency 3_norm,BUR_CREDIT_CURRENCY_currency 4_count,BUR_CREDIT_CURRENCY_currency 4_norm,BUR_CREDIT_TYPE_Another type of loan_count,BUR_CREDIT_TYPE_Another type of loan_norm,BUR_CREDIT_TYPE_Car loan_count,BUR_CREDIT_TYPE_Car loan_norm,BUR_CREDIT_TYPE_Cash loan (non-earmarked)_count,BUR_CREDIT_TYPE_Cash loan (non-earmarked)_norm,BUR_CREDIT_TYPE_Consumer credit_count,BUR_CREDIT_TYPE_Consumer credit_norm,BUR_CREDIT_TYPE_Credit card_count,BUR_CREDIT_TYPE_Credit card_norm,BUR_CREDIT_TYPE_Interbank credit_count,BUR_CREDIT_TYPE_Interbank credit_norm,BUR_CREDIT_TYPE_Loan for business development_count,BUR_CREDIT_TYPE_Loan for business development_norm,BUR_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count,BUR_CREDIT_TYPE_Loan for purchase of shares (margin lending)_norm,BUR_CREDIT_TYPE_Loan for the purchase of equipment_count,BUR_CREDIT_TYPE_Loan for the purchase of equipment_norm,BUR_CREDIT_TYPE_Loan for working capital replenishment_count,BUR_CREDIT_TYPE_Loan for working capital replenishment_norm,BUR_CREDIT_TYPE_Microloan_count,BUR_CREDIT_TYPE_Microloan_norm,BUR_CREDIT_TYPE_Mobile operator loan_count,BUR_CREDIT_TYPE_Mobile operator loan_norm,BUR_CREDIT_TYPE_Mortgage_count,BUR_CREDIT_TYPE_Mortgage_norm,BUR_CREDIT_TYPE_Real estate loan_count,BUR_CREDIT_TYPE_Real estate loan_norm,BUR_CREDIT_TYPE_Unknown type of loan_count,BUR_CREDIT_TYPE_Unknown type of loan_norm
0,0,1,1.00000,0,0.00000,0,0.00000,0,0.00000,1,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,1,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
1,1,1,1.00000,0,0.00000,0,0.00000,0,0.00000,1,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,1,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
2,2,2,0.33333,0,0.00000,4,0.66667,0,0.00000,6,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,6,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
3,3,3,0.42857,0,0.00000,4,0.57143,0,0.00000,7,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,6,0.85714,1,0.14286,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
4,4,3,0.50000,0,0.00000,3,0.50000,0,0.00000,6,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,5,0.83333,1,0.16667,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000


__4. Handling numerical features__

In [17]:
bu_count = bu.groupby("SK_ID_CURR")['SK_ID_BUREAU'].count().reset_index()
bu_count = bu_count.rename(columns={'SK_ID_BUREAU': 'CNT_BUREAU'})
bu_count.head()

,SK_ID_CURR,CNT_BUREAU
0,0,1
1,1,1
2,2,6
3,3,7
4,4,6


In [18]:
bu_num = ft.sub_num_agg(bu,'SK_ID_CURR','BUR')
bu_num.head()

,SK_ID_CURR,BUR_DAYS_CREDIT_min,BUR_DAYS_CREDIT_max,BUR_DAYS_CREDIT_mean,BUR_DAYS_CREDIT_sum,BUR_CREDIT_DAY_OVERDUE_min,BUR_CREDIT_DAY_OVERDUE_max,BUR_CREDIT_DAY_OVERDUE_mean,BUR_CREDIT_DAY_OVERDUE_sum,BUR_DAYS_CREDIT_ENDDATE_min,BUR_DAYS_CREDIT_ENDDATE_max,BUR_DAYS_CREDIT_ENDDATE_mean,BUR_DAYS_CREDIT_ENDDATE_sum,BUR_DAYS_ENDDATE_FACT_min,BUR_DAYS_ENDDATE_FACT_max,BUR_DAYS_ENDDATE_FACT_mean,BUR_DAYS_ENDDATE_FACT_sum,BUR_AMT_CREDIT_MAX_OVERDUE_min,BUR_AMT_CREDIT_MAX_OVERDUE_max,BUR_AMT_CREDIT_MAX_OVERDUE_mean,BUR_AMT_CREDIT_MAX_OVERDUE_sum,BUR_CNT_CREDIT_PROLONG_min,BUR_CNT_CREDIT_PROLONG_max,BUR_CNT_CREDIT_PROLONG_mean,BUR_CNT_CREDIT_PROLONG_sum,BUR_AMT_CREDIT_SUM_min,BUR_AMT_CREDIT_SUM_max,BUR_AMT_CREDIT_SUM_mean,BUR_AMT_CREDIT_SUM_sum,BUR_AMT_CREDIT_SUM_DEBT_min,BUR_AMT_CREDIT_SUM_DEBT_max,BUR_AMT_CREDIT_SUM_DEBT_mean,BUR_AMT_CREDIT_SUM_DEBT_sum,BUR_AMT_CREDIT_SUM_LIMIT_min,BUR_AMT_CREDIT_SUM_LIMIT_max,BUR_AMT_CREDIT_SUM_LIMIT_mean,BUR_AMT_CREDIT_SUM_LIMIT_sum,BUR_AMT_CREDIT_SUM_OVERDUE_min,BUR_AMT_CREDIT_SUM_OVERDUE_max,BUR_AMT_CREDIT_SUM_OVERDUE_mean,BUR_AMT_CREDIT_SUM_OVERDUE_sum,BUR_DAYS_CREDIT_UPDATE_min,BUR_DAYS_CREDIT_UPDATE_max,BUR_DAYS_CREDIT_UPDATE_mean,BUR_DAYS_CREDIT_UPDATE_sum,BUR_AMT_ANNUITY_min,BUR_AMT_ANNUITY_max,BUR_AMT_ANNUITY_mean,BUR_AMT_ANNUITY_sum,BUR_BU_FLAG_DPD_min,BUR_BU_FLAG_DPD_max,BUR_BU_FLAG_DPD_mean,BUR_BU_FLAG_DPD_sum,BUR_FLAG_PROLONG_min,BUR_FLAG_PROLONG_max,BUR_FLAG_PROLONG_mean,BUR_FLAG_PROLONG_sum
0,0,-63,-63,-63.00000,-63,0,0,0.00000,0,237.00000,237.00000,237.00000,237.00000,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,0.00000,0,0,0.00000,0,85513.50000,85513.50000,85513.50000,85513.50000,77566.50000,77566.50000,77566.50000,77566.50000,NaN,NaN,NaN,0.00000,0.00000,0.00000,0.00000,0.00000,-28,-28,-28.00000,-28,NaN,NaN,NaN,0.00000,0,0,0.00000,0,0,0,0.00000,0
1,1,-2348,-2348,-2348.00000,-2348,0,0,0.00000,0,-2044.00000,-2044.00000,-2044.00000,-2044.00000,NaN,NaN,NaN,0.00000,11666.38500,11666.38500,11666.38500,11666.38500,0,0,0.00000,0,28350.00000,28350.00000,28350.00000,28350.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-18,-18,-18.00000,-18,NaN,NaN,NaN,0.00000,0,0,0.00000,0,0,0,0.00000,0
2,2,-2901,-30,-810.33333,-4862,0,0,0.00000,0,-2597.00000,274.00000,-572.16667,-3433.00000,-2567.00000,-155.00000,-1014.00000,-4056.00000,0.00000,7543.98000,3771.99000,7543.98000,0,0,0.00000,0,20656.08000,86445.00000,42390.18000,254341.08000,0.00000,58333.50000,16069.50000,96417.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-2562,-7,-625.50000,-3753,NaN,NaN,NaN,0.00000,0,0,0.00000,0,0,0,0.00000,0
3,3,-2865,-116,-1131.42857,-7920,0,0,0.00000,0,-2683.00000,502.00000,-658.71429,-4611.00000,-2683.00000,-189.00000,-1394.00000,-5576.00000,0.00000,0.00000,0.00000,0.00000,0,0,0.00000,0,19305.00000,148500.00000,69136.07143,483952.50000,0.00000,157698.00000,28395.64286,198769.50000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-2682,-7,-805.00000,-5635,NaN,NaN,NaN,0.00000,0,0,0.00000,0,0,0,0.00000,0
4,4,-1056,-313,-773.33333,-4640,0,0,0.00000,0,-509.00000,1148.00000,474.50000,2847.00000,-965.00000,-398.00000,-761.00000,-2283.00000,NaN,NaN,NaN,0.00000,0,0,0.00000,0,112500.00000,945000.00000,470424.00000,2822544.00000,0.00000,655510.50000,209084.25000,1254505.50000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-104,-6,-63.33333,-380,NaN,NaN,NaN,0.00000,0,0,0.00000,0,0,0,0.00000,0


__4. Merging__

In [19]:
bu_merge = bu_count.merge(bu_cat, on='SK_ID_CURR', how='left')
bu_merge = bu_merge.merge(bu_num, on='SK_ID_CURR', how='left')
bu_merge.head()

,SK_ID_CURR,CNT_BUREAU,BUR_CREDIT_ACTIVE_Active_count,BUR_CREDIT_ACTIVE_Active_norm,BUR_CREDIT_ACTIVE_Bad debt_count,BUR_CREDIT_ACTIVE_Bad debt_norm,BUR_CREDIT_ACTIVE_Closed_count,BUR_CREDIT_ACTIVE_Closed_norm,BUR_CREDIT_ACTIVE_Sold_count,BUR_CREDIT_ACTIVE_Sold_norm,BUR_CREDIT_CURRENCY_currency 1_count,BUR_CREDIT_CURRENCY_currency 1_norm,BUR_CREDIT_CURRENCY_currency 2_count,BUR_CREDIT_CURRENCY_currency 2_norm,BUR_CREDIT_CURRENCY_currency 3_count,BUR_CREDIT_CURRENCY_currency 3_norm,BUR_CREDIT_CURRENCY_currency 4_count,BUR_CREDIT_CURRENCY_currency 4_norm,BUR_CREDIT_TYPE_Another type of loan_count,BUR_CREDIT_TYPE_Another type of loan_norm,BUR_CREDIT_TYPE_Car loan_count,BUR_CREDIT_TYPE_Car loan_norm,BUR_CREDIT_TYPE_Cash loan (non-earmarked)_count,BUR_CREDIT_TYPE_Cash loan (non-earmarked)_norm,BUR_CREDIT_TYPE_Consumer credit_count,BUR_CREDIT_TYPE_Consumer credit_norm,BUR_CREDIT_TYPE_Credit card_count,BUR_CREDIT_TYPE_Credit card_norm,BUR_CREDIT_TYPE_Interbank credit_count,BUR_CREDIT_TYPE_Interbank credit_norm,BUR_CREDIT_TYPE_Loan for business development_count,BUR_CREDIT_TYPE_Loan for business development_norm,BUR_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count,BUR_CREDIT_TYPE_Loan for purchase of shares (margin lending)_norm,BUR_CREDIT_TYPE_Loan for the purchase of equipment_count,BUR_CREDIT_TYPE_Loan for the purchase of equipment_norm,BUR_CREDIT_TYPE_Loan for working capital replenishment_count,BUR_CREDIT_TYPE_Loan for working capital replenishment_norm,BUR_CREDIT_TYPE_Microloan_count,BUR_CREDIT_TYPE_Microloan_norm,BUR_CREDIT_TYPE_Mobile operator loan_count,BUR_CREDIT_TYPE_Mobile operator loan_norm,BUR_CREDIT_TYPE_Mortgage_count,BUR_CREDIT_TYPE_Mortgage_norm,BUR_CREDIT_TYPE_Real estate loan_count,BUR_CREDIT_TYPE_Real estate loan_norm,BUR_CREDIT_TYPE_Unknown type of loan_count,BUR_CREDIT_TYPE_Unknown type of loan_norm,BUR_DAYS_CREDIT_min,BUR_DAYS_CREDIT_max,BUR_DAYS_CREDIT_mean,BUR_DAYS_CREDIT_sum,BUR_CREDIT_DAY_OVERDUE_min,BUR_CREDIT_DAY_OVERDUE_max,BUR_CREDIT_DAY_OVERDUE_mean,BUR_CREDIT_DAY_OVERDUE_sum,BUR_DAYS_CREDIT_ENDDATE_min,BUR_DAYS_CREDIT_ENDDATE_max,BUR_DAYS_CREDIT_ENDDATE_mean,BUR_DAYS_CREDIT_ENDDATE_sum,BUR_DAYS_ENDDATE_FACT_min,BUR_DAYS_ENDDATE_FACT_max,BUR_DAYS_ENDDATE_FACT_mean,BUR_DAYS_ENDDATE_FACT_sum,BUR_AMT_CREDIT_MAX_OVERDUE_min,BUR_AMT_CREDIT_MAX_OVERDUE_max,BUR_AMT_CREDIT_MAX_OVERDUE_mean,BUR_AMT_CREDIT_MAX_OVERDUE_sum,BUR_CNT_CREDIT_PROLONG_min,BUR_CNT_CREDIT_PROLONG_max,BUR_CNT_CREDIT_PROLONG_mean,BUR_CNT_CREDIT_PROLONG_sum,BUR_AMT_CREDIT_SUM_min,BUR_AMT_CREDIT_SUM_max,BUR_AMT_CREDIT_SUM_mean,BUR_AMT_CREDIT_SUM_sum,BUR_AMT_CREDIT_SUM_DEBT_min,BUR_AMT_CREDIT_SUM_DEBT_max,BUR_AMT_CREDIT_SUM_DEBT_mean,BUR_AMT_CREDIT_SUM_DEBT_sum,BUR_AMT_CREDIT_SUM_LIMIT_min,BUR_AMT_CREDIT_SUM_LIMIT_max,BUR_AMT_CREDIT_SUM_LIMIT_mean,BUR_AMT_CREDIT_SUM_LIMIT_sum,BUR_AMT_CREDIT_SUM_OVERDUE_min,BUR_AMT_CREDIT_SUM_OVERDUE_max,BUR_AMT_CREDIT_SUM_OVERDUE_mean,BUR_AMT_CREDIT_SUM_OVERDUE_sum,BUR_DAYS_CREDIT_UPDATE_min,BUR_DAYS_CREDIT_UPDATE_max,BUR_DAYS_CREDIT_UPDATE_mean,BUR_DAYS_CREDIT_UPDATE_sum,BUR_AMT_ANNUITY_min,BUR_AMT_ANNUITY_max,BUR_AMT_ANNUITY_mean,BUR_AMT_ANNUITY_sum,BUR_BU_FLAG_DPD_min,BUR_BU_FLAG_DPD_max,BUR_BU_FLAG_DPD_mean,BUR_BU_FLAG_DPD_sum,BUR_FLAG_PROLONG_min,BUR_FLAG_PROLONG_max,BUR_FLAG_PROLONG_mean,BUR_FLAG_PROLONG_sum
0,0,1,1,1.00000,0,0.00000,0,0.00000,0,0.00000,1,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,1,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,-63,-63,-63.00000,-63,0,0,0.00000,0,237.00000,237.00000,237.00000,237.00000,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,0.00000,0,0,0.00000,0,85513.50000,85513.50000,85513.50000,85513.50000,77566.50000,77566.50000,77566.50000,77566.50000,NaN,NaN,NaN,0.00000,0.00000,0.00000,0.00000,0.00000,-28,-28,-28.00000,-28,NaN,NaN,NaN,0.00000,0,0,0.00000,0,0,0,0.00000,0
1,1,1,1,1.00000,0,0.00000,0,0.00000,0,0.00000,1,1.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.

### bureau_balance

__1. Data Exploration__

In [20]:
bb = bubal_raw.copy()

In [21]:
ft.data_explore(bb)

,info
Rows,27299925
Features,3
Duplicate Rows,0
int64,2
object,1


In [22]:
ft.check_nan(bb)

,nan,%nan


In [23]:
ft.multi_features_explore(bb)

index,dtype,nonnull,%nonnull,nan,%nan,nunique,nunique_nan,max,min,mean,std,unique,frequency,%value,most
SK_ID_BUREAU,int64,27299925,1.00000,0,0.00000,817395,817395,6842888,5001709,6036297.33297,492348.85690,NaN,NaN,NaN,NaN
MONTHS_BALANCE,int64,27299925,1.00000,0,0.00000,97,97,0,-96,-30.74169,23.86451,NaN,NaN,NaN,NaN
STATUS,object,27299925,1.00000,0,0.00000,8,8,NaN,NaN,NaN,NaN,"[C, 0, X, 1, 2, 3, 5, 4]","{'C': 13646993, '0': 7499507, 'X': 5810482, '1': 242347, '5': 62406, '2': 23419, '3': 8924, '4': 5847}","{'C': 0.5, '0': 0.27, 'X': 0.21, '1': 0.01, '5': 0.0, '2': 0.0, '3': 0.0, '4': 0.0}",[C]


__2. Dealing with categorical features__

In [26]:
# merge with bureau balance to get SK_ID_CURR
bb = bu[['SK_ID_CURR','SK_ID_BUREAU']].merge(bb, on = 'SK_ID_BUREAU', how = 'left')

In [27]:
bb_cat = ft.sub_cate_norm(bb,'SK_ID_CURR', 'BB').reset_index()
bb_cat.head()

,SK_ID_CURR,BB_STATUS_0_count,BB_STATUS_0_norm,BB_STATUS_1_count,BB_STATUS_1_norm,BB_STATUS_2_count,BB_STATUS_2_norm,BB_STATUS_3_count,BB_STATUS_3_norm,BB_STATUS_4_count,BB_STATUS_4_norm,BB_STATUS_5_count,BB_STATUS_5_norm,BB_STATUS_C_count,BB_STATUS_C_norm,BB_STATUS_X_count,BB_STATUS_X_norm
0,0,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
1,1,10,0.18519,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,44,0.81481,0,0.00000
2,2,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
3,3,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
4,4,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000


__3. Dealing with numerical features__

In [29]:
bb_count = bb.groupby("SK_ID_CURR")['SK_ID_BUREAU'].agg(['count']).reset_index()
bb_count = bb_count.rename(columns={"count": "BB_CNT_MONTHS"})
bb_count.head()

,SK_ID_CURR,BB_CNT_MONTHS
0,0,1
1,1,54
2,2,6
3,3,7
4,4,6


In [30]:
bb_num = ft.sub_cate_norm(bb, 'SK_ID_CURR', 'BB')
bb_num.head()

,BB_STATUS_0_count,BB_STATUS_0_norm,BB_STATUS_1_count,BB_STATUS_1_norm,BB_STATUS_2_count,BB_STATUS_2_norm,BB_STATUS_3_count,BB_STATUS_3_norm,BB_STATUS_4_count,BB_STATUS_4_norm,BB_STATUS_5_count,BB_STATUS_5_norm,BB_STATUS_C_count,BB_STATUS_C_norm,BB_STATUS_X_count,BB_STATUS_X_norm
SK_ID_CURR,,,,,,,,,,,,,,,,
0,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
1,10,0.18519,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,44,0.81481,0,0.00000
2,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
3,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
4,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000


__4. Merging__

In [31]:
bb_merge = bb_count.merge(bb_cat, on='SK_ID_CURR', how='left')
bb_merge = bb_merge.merge(bb_num, on='SK_ID_CURR', how='left')
bb_merge.head()

,SK_ID_CURR,BB_CNT_MONTHS,BB_STATUS_0_count_x,BB_STATUS_0_norm_x,BB_STATUS_1_count_x,BB_STATUS_1_norm_x,BB_STATUS_2_count_x,BB_STATUS_2_norm_x,BB_STATUS_3_count_x,BB_STATUS_3_norm_x,BB_STATUS_4_count_x,BB_STATUS_4_norm_x,BB_STATUS_5_count_x,BB_STATUS_5_norm_x,BB_STATUS_C_count_x,BB_STATUS_C_norm_x,BB_STATUS_X_count_x,BB_STATUS_X_norm_x,BB_STATUS_0_count_y,BB_STATUS_0_norm_y,BB_STATUS_1_count_y,BB_STATUS_1_norm_y,BB_STATUS_2_count_y,BB_STATUS_2_norm_y,BB_STATUS_3_count_y,BB_STATUS_3_norm_y,BB_STATUS_4_count_y,BB_STATUS_4_norm_y,BB_STATUS_5_count_y,BB_STATUS_5_norm_y,BB_STATUS_C_count_y,BB_STATUS_C_norm_y,BB_STATUS_X_count_y,BB_STATUS_X_norm_y
0,0,1,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
1,1,54,10,0.18519,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,44,0.81481,0,0.00000,10,0.18519,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,44,0.81481,0,0.00000
2,2,6,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
3,3,7,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000
4,4,6,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000


### bureau + bureau_balance

In [32]:
bu_full = bu_merge.merge(bb_merge, on = 'SK_ID_CURR', how = 'left')

__1. Scaling__

In [33]:
bu_scaled = bu_full.copy()
bu_scaled.iloc[:, 1:] = ft.scale_df_full(bu_scaled.iloc[:,1:], method='standard')

__2. Filling missing values__

In [34]:
# merge with SK_ID_CURR and TARGET of the train/test to fillna in the whole set
bu_final = id_target.merge(bu_scaled, on='SK_ID_CURR', how='left')

# Fillna with 0
bu_final.iloc[:, 2:] = ft.fillna(bu_final.iloc[:, 2:], 'constant', 0)

__3. Saving files__

In [35]:
bu_final.to_csv("prepared_files/dp_2bu.csv")